In [1]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex
import libsbml
import pandas as pd
import copy


In [2]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6


In [3]:
#Solver
model.solver = 'cplex'

In [4]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)

#First initialize all inputs to approximate infinity (1e6)
for imports in model.exchanges:
    if imports.upper_bound == 1000:
        model.reactions.get_by_id(imports.id).upper_bound = inf
    elif imports.lower_bound == -1000:
        print(imports.id, imports.lower_bound)
        model.reactions.get_by_id(imports.id).lower_bound = -inf
        print(imports.id, imports.lower_bound)
        
mm.read_medium_csv('../misc/photo_medium.csv', model)

#Test natin kung magwowork sa est. infinite
#Yes setting bounds to 1000 works
model_media = model.medium
for items in model.exchanges:
    print(items.id, items.bounds)



EX_co2_LPAREN_e_RPAREN__M -1000.0
EX_co2_LPAREN_e_RPAREN__M -1000000.0
EX_no3_LPAREN_e_RPAREN__M -1000.0
EX_no3_LPAREN_e_RPAREN__M -1000000.0
EX_fru_DASH_B_LPAREN_e_RPAREN__M -1000.0
EX_fru_DASH_B_LPAREN_e_RPAREN__M -1000000.0
EX_glc_DASH_A_LPAREN_e_RPAREN__M -1000.0
EX_glc_DASH_A_LPAREN_e_RPAREN__M -1000000.0
EX_hco3_LPAREN_e_RPAREN__M -1000.0
EX_hco3_LPAREN_e_RPAREN__M -1000000.0
EX_so3_LPAREN_e_RPAREN__M -1000.0
EX_so3_LPAREN_e_RPAREN__M -1000000.0
EX_h2s_LPAREN_e_RPAREN__M -1000.0
EX_h2s_LPAREN_e_RPAREN__M -1000000.0
EX_nh4_LPAREN_e_RPAREN__M -1000.0
EX_nh4_LPAREN_e_RPAREN__M -1000000.0
EX_tsul_LPAREN_e_RPAREN__M -1000.0
EX_tsul_LPAREN_e_RPAREN__M -1000000.0
EX_fe3_LPAREN_e_RPAREN__M -1000.0
EX_fe3_LPAREN_e_RPAREN__M -1000000.0
EX_photonVis_LPAREN_e_RPAREN__M -1000.0
EX_photonVis_LPAREN_e_RPAREN__M -1000000.0
EX_co2_LPAREN_e_RPAREN__BS -1000.0
EX_co2_LPAREN_e_RPAREN__BS -1000000.0
EX_no3_LPAREN_e_RPAREN__BS -1000.0
EX_no3_LPAREN_e_RPAREN__BS -1000000.0
EX_sucr_LPAREN_e_RPAREN__BS -

In [30]:
#Modifications to medium

model_media['EX_photonVis_LPAREN_e_RPAREN__M'] = 200 #PPFD used in Ermakova et al for high light
model_media['EX_photonVis_LPAREN_e_RPAREN__BS'] = 200 #PPFD used in Ermakova et al for high light 

#Set CO2 bounds to ~29 m-2 s-1
model_media['EX_co2_LPAREN_e_RPAREN__M'] = 30
model_media['EX_co2_LPAREN_e_RPAREN__BS'] = 30
# model_media['EX_o2_LPAREN_e_RPAREN__M'] = inf
# model_media['EX_o2_LPAREN_e_RPAREN__BS'] = inf

model.medium = model_media

In [6]:
#Add constraints to model

#BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
BS_photon_import = model.reactions.get_by_id("EX_photonVis_LPAREN_e_RPAREN__BS")
M_photon_import = model.reactions.get_by_id("EX_photonVis_LPAREN_e_RPAREN__M")

photon_flux = model.problem.Constraint(M_photon_import.flux_expression 
                                       - BS_photon_import.flux_expression,
                                       lb = 0, ub = 1000)
model.add_cons_vars(photon_flux)


#Note: CO2 uptake is not restricted in BS cells but will be constrained to be  equal/less than M cell
BS_co2_import = model.reactions.get_by_id("EX_co2_LPAREN_e_RPAREN__BS") 
M_co2_import = model.reactions.get_by_id("EX_co2_LPAREN_e_RPAREN__M")
co2_flux_cons = model.problem.Constraint(M_co2_import.flux_expression - BS_co2_import.flux_expression, lb = 0, ub = inf)
model.add_cons_vars(co2_flux_cons)

#CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
co2_import_cons = model.problem.Constraint(M_co2_import.flux_expression + BS_co2_import.flux_expression, lb = -30.2, ub = -27.8)
model.add_cons_vars(co2_import_cons)
#Oks na
#CO2 intake needs to be mostly in the M cells. Not sure of the exact value: 
#try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
#I'll assume that the ratio is at 9.5:0.5 in the meantime, meaning that there is minimal gas exchange into the BS cell


#Automatic na palang nalalagay siya as constraint

#First is the BS and M Cell constraints

In [7]:
co2_intake_ratio = mm.set_fix_flux_ratio({M_co2_import.id:9, BS_co2_import.id:1},model)

model.remove_cons_vars(co2_intake_ratio)

In [8]:
#What if I restrict yung import ng O2?

M_o2_import = model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN__M')
BS_o2_import = model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN__BS')
#Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
#Actually this value is also reflected in Von Caemmerer's "Modelling C3 metabolism" (pg. 85)
o2_import_cons = model.problem.Constraint(M_o2_import.flux_expression + BS_o2_import.flux_expression, lb = -3.312, ub = 0)
model.add_cons_vars(o2_import_cons)


In [31]:
#Add enzyme constraints

#PEPC
wt_pepc = 2.4 #umol m-2 s-1
wt_mdh = 11.18 #umol m-2 s-1
wt_nadp_me = 0.14 #umol m-2 s-1
wt_ppdk = 0.31 #umol m-2 s-1
wt_CA = 15.74 #umol m-2  s-1

#PEPC constraint (Reaction id: PPCc)
pepc_BS = model.reactions.PPCc_BS
pepc_M = model.reactions.PPCc_M

wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                        + pepc_M.flux_expression, 
                                        lb = wt_pepc, ub = wt_pepc)
model.add_cons_vars(wt_pepc_cons)

#PPDK constraints (Reaction id: PPDKS) (note that this is found in the chloroplast?) 
#Not detected via immunolocalization but enzyme activity is detected
ppdks_BS = model.reactions.PPDKs_BS
ppdks_M = model.reactions.PPDKs_M

wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                         + ppdks_M.flux_expression, 
                                         lb = wt_ppdk, ub = wt_ppdk)
model.add_cons_vars(wt_ppdks_cons)

#Malate Dehydrogenase 
mdh2c_M = model.reactions.MDH2c_M
mdh2c_BS = model.reactions.MDH2c_BS
mdh2s_M = model.reactions.MDH2s_M
mdh2s_BS = model.reactions.MDH2s_BS

wt_mdh_cons = model.problem.Constraint(mdh2c_M.flux_expression 
                                       + mdh2c_BS.flux_expression 
                                       + mdh2s_M.flux_expression 
                                       + mdh2s_BS.flux_expression, 
                                       lb= wt_mdh, ub=wt_mdh)
model.add_cons_vars(wt_mdh_cons)


#NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
# nadp_me_M = model.reactions.MDH2s_M
# nadp_me_BS = model.reactions.MDH2s_BS

# wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
#                                          + nadp_me_BS.flux_expression,
#                                          lb= wt_nadp_me, ub=wt_nadp_me)
# model.add_cons_vars(wt_nadpme_cons)
#Weird, it causes the activity of RBCL to significantly vary from the actual values

#Rbcl constaints

In [32]:
sample_fluxes = cobra.flux_analysis.pfba(model)
sample_fluxes

,fluxes,reduced_costs
Coleoptile_Biomass_M,0.000000,1.943345e+05
Straw_Biomass_M,1.506190,-2.000000e+00
ACCOAC_OSAc_M,0.000000,-2.000000e+00
ACCOAC_OSAr_M,0.152068,-2.000000e+00
ACPC_OSs_M,0.000000,-2.000000e+00
...,...,...
g3p_pd,0.000000,-1.182343e-11
3pg_pd,0.000000,-1.364242e-11
glyclt_pd,0.000000,2.000000e+00
acon_DASH_C_pd,0.000000,2.000000e+00


In [34]:
#This is cool, the M cell is similar to the WT rate of 83.59+-5.63 :)
print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPC2s_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPC2s_BS'])




rbcl M cell:  83.36015712631428 rbcl BS cell:  60.65618166168781
rbcl M cell (photorespiration) 0.0 rbcl BS cell (PR) 0.0


In [26]:
for items in model.reactions:
    print(items.id, sample_fluxes[items.id])

Coleoptile_Biomass_M 0.0
Straw_Biomass_M 3.470831393348682
ACCOAC_OSAc_M 0.0
ACCOAC_OSAr_M 0.3504220791352697
ACPC_OSs_M 0.0
ACPC_OSm_M 0.0
FAC_Osc_M 0.0
FAC_Osr_M -0.009891869471043745
EX_co2_LPAREN_e_RPAREN__M -0.1999999999999993
EX_h2o_LPAREN_e_RPAREN__M 489.0430065702915
EX_h_LPAREN_e_RPAREN__M -2000.0
EX_no3_LPAREN_e_RPAREN__M 0.0
EX_o2_LPAREN_e_RPAREN__M -3.312
EX_pi_LPAREN_e_RPAREN__M -1.7805470493637852
EX_sucr_LPAREN_e_RPAREN__M 0.0
EX_fru_DASH_B_LPAREN_e_RPAREN__M 0.0
EX_glc_DASH_A_LPAREN_e_RPAREN__M 0.0
EX_so4_LPAREN_e_RPAREN__M -0.09404768702752457
EX_hco3_LPAREN_e_RPAREN__M 0.0
EX_so3_LPAREN_e_RPAREN__M 0.0
EX_h2s_LPAREN_e_RPAREN__M 0.0
EX_nh4_LPAREN_e_RPAREN__M 0.0
EX_asn_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_gln_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_etoh_LPAREN_e_RPAREN__M -0.0
EX_ac_LPAREN_e_RPAREN__M 0.0
EX_lac_DASH_L_LPAREN_e_RPAREN__M -0.0
EX_tsul_LPAREN_e_RPAREN__M 0.0
EX_fe2_LPAREN_e_RPAREN__M 0.0
EX_fe3_LPAREN_e_RPAREN__M 0.0
EX_mg2_LPAREN_e_RPAREN__M 0.0
EX_ala_DASH_L_LPAR

DM_CHSTEROL_LPAREN_c_RPAREN__M -0.0
DM_CHUMU_LPAREN_c_RPAREN__M -0.0
DM_CNDM35DG_LPAREN_c_RPAREN__M -0.0
DM_COA_LPAREN_c_RPAREN__M 3.5042207910562784e-05
DM_COUMN_LPAREN_c_RPAREN__M -0.0
DM_COUMOH_LPAREN_c_RPAREN__M 0.0
DM_CPP1_LPAREN_s_RPAREN__M -0.0
DM_CRPE_LPAREN_c_RPAREN__M -0.0
DM_CYARTFER_LPAREN_c_RPAREN__M -0.0
DM_CYTC_LPAREN_m_RPAREN__M -0.0
DM_CYTC_LPAREN_s_RPAREN__M -0.0
DM_C_DASH_ZTN7G_LPAREN_c_RPAREN__M -0.0
DM_C_DASH_ZTN9G_LPAREN_c_RPAREN__M -0.0
DM_ZTNG_LPAREN_c_RPAREN__M -0.0
DM_DHBZCAROT_LPAREN_s_RPAREN__M -0.0
DM_DHZ7G_LPAREN_c_RPAREN__M -0.0
DM_DHZ9G_LPAREN_c_RPAREN__M -0.0
DM_DHZG_LPAREN_c_RPAREN__M -0.0
DM_DVCHLDA_LPAREN_s_RPAREN__M -0.0
DM_EIHKAUR_LPAREN_c_RPAREN__M -0.0
DM_ELM_LPAREN_c_RPAREN__M -0.0
DM_ENTB_LPAREN_s_RPAREN__M -0.0
DM_EPCTCN_LPAREN_c_RPAREN__M -0.0
DM_EPLZN_LPAREN_c_RPAREN__M -0.0
DM_ERGTOL_LPAREN_c_RPAREN__M -0.0
DM_ETHYLENE_LPAREN_c_RPAREN__M -0.0
DM_GA13_LPAREN_c_RPAREN__M -0.0
DM_GA17_LPAREN_c_RPAREN__M -0.0
DM_GA29CAT_LPAREN_c_RPAREN__M -0.0


PHENMUTc_BS 0.0
DRPAc_BS 0.0
UDPGALMc_BS 0.0
THD2c_BS 0.0
MALTDPs_BS 0.0
MALTGAMs_BS 0.0
NRGNMTc_BS 0.0
UDPGLTLNGc_BS 0.0
UDPRMNLc_BS 0.0
UDPGAPGNGc_BS 0.0
UDPRMNAc_BS 0.0
CRPESc_BS 0.0
HUMLSc_BS 0.0
ELMSc_BS 0.0
GAAKORG4498c_BS 0.0
NADtm_BS 0.0
UACGAMtm_BS 0.0
CKDOtm_BS 0.0
THFtm_BS 0.0
FORtm_BS 0.0
ADENm_BS 0.0
PYRt2m_BS -11.428294340553848
ASPt2m_BS 0.0
CYSt2m_BS 0.0
THRt2m_BS 0.0
LYSt2m_BS 0.0
METt2m_BS 0.0
PHEt2m_BS 0.0
ILEt2m_BS 0.0
VALt2m_BS 0.2367245119410182
TYRt2m_BS 0.0
PROt2m_BS 0.0
HISt2m_BS 0.0
LEUt2m_BS 0.0
TRPt2m_BS 0.0
ASPGLU2m_BS 0.0
ALAt2m_BS -11.901743364435884
H2Otm_BS -940.1399425825751
CO2tm_BS 10.07993541766028
NH4tm_BS 11.500179341370913
O2tmr_BS 103.11639626074088
FADH2tm_BS 0.0
FADtm_BS 0.0
PItm_BS 656.0373016536877
H2O2tm_BS 0.0
ATPtm_BS 656.0373016536877
4ABUTtmi_BS 0.0
SUCFUMtmr_BS 42.8820395360559
GLYtm_BS 33.86907547328259
SERtm_BS -10.703877279416812
DICtm_BS 42.8820395360559
MALCITtm_BS 0.0
MALAKGtm_BS 0.3780371835572325
SO4AKGtm_BS 0.0
AKGCITtm_BS 0.0

In [27]:
#Exchange reactions
for exs in model.exchanges:
    print(exs.id, sample_fluxes[exs.id])

EX_co2_LPAREN_e_RPAREN__M -0.1999999999999993
EX_h2o_LPAREN_e_RPAREN__M 489.0430065702915
EX_h_LPAREN_e_RPAREN__M -2000.0
EX_no3_LPAREN_e_RPAREN__M 0.0
EX_o2_LPAREN_e_RPAREN__M -3.312
EX_pi_LPAREN_e_RPAREN__M -1.7805470493637852
EX_sucr_LPAREN_e_RPAREN__M 0.0
EX_fru_DASH_B_LPAREN_e_RPAREN__M 0.0
EX_glc_DASH_A_LPAREN_e_RPAREN__M 0.0
EX_so4_LPAREN_e_RPAREN__M -0.09404768702752457
EX_hco3_LPAREN_e_RPAREN__M 0.0
EX_so3_LPAREN_e_RPAREN__M 0.0
EX_h2s_LPAREN_e_RPAREN__M 0.0
EX_nh4_LPAREN_e_RPAREN__M 0.0
EX_asn_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_gln_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_etoh_LPAREN_e_RPAREN__M -0.0
EX_ac_LPAREN_e_RPAREN__M 0.0
EX_lac_DASH_L_LPAREN_e_RPAREN__M -0.0
EX_tsul_LPAREN_e_RPAREN__M 0.0
EX_fe2_LPAREN_e_RPAREN__M 0.0
EX_fe3_LPAREN_e_RPAREN__M 0.0
EX_mg2_LPAREN_e_RPAREN__M 0.0
EX_ala_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_arg_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_asp_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_cys_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_glu_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_gly_LPAREN_e_RPA

In [14]:
sample_fluxes['HCO3Es_BS']

0.0

In [28]:
#Show all pd reactions
pd_rxns = [rxn for rxn in model.reactions if "_pd" in rxn.id]

for items in pd_rxns:
    print(items.id, sample_fluxes[items.id])

ala_DASH_L_pd 0.0
arg_DASH_L_pd 0.0
asp_DASH_L_pd 0.0
cys_DASH_L_pd 0.0
glu_DASH_L_pd 0.0
gly_pd 0.0
his_DASH_L_pd 0.0
ile_DASH_L_pd 0.0
leu_DASH_L_pd 0.0
lys_DASH_L_pd 0.0
met_DASH_L_pd 0.0
phe_DASH_L_pd 0.0
pro_DASH_L_pd 0.0
ser_DASH_L_pd 0.0
thr_DASH_L_pd 0.0
trp_DASH_L_pd 0.0
tyr_DASH_L_pd 0.0
val_DASH_L_pd 0.0
glc_DASH_A_pd 0.0
sucr_pd 0.0
fru_DASH_B_pd 0.0
udparab_pd 0.0
udpxyl_pd 0.0
udpgal_pd 0.0
udpg_pd 0.0
udpglcur_pd 0.0
co2_pd 0.0
h2o_pd 0.0
o2_pd 0.0
pep_pd 0.0
pyr_pd 0.0
mal_DASH_L_pd 0.0
accoa_pd 0.0
glu5p_pd 0.0
akg_pd 0.0
icit_pd 0.0
gal_pd 0.0
g1p_pd 0.0
tre_pd 0.0
glc_DASH_B_pd 0.0
tre6p_pd 0.0
g6p_DASH_A_pd 0.0
b_DASH_glucan_pd 0.0
mnt_pd 0.0
melib_pd 0.0
stc_pd 0.0
raffin_pd 0.0
1Dgali_pd 0.0
inost_pd 0.0
cit_pd 0.0
oxa_pd 0.0
2pg_pd 0.0
dhap_pd 0.0
g3p_pd 0.0
3pg_pd 0.0
glyclt_pd 0.0
acon_DASH_C_pd 0.0
succ_pd 0.0


In [29]:
#Let's try to do initial FBA
model.metabolites.gly_c0.summary()


Percent,Flux,Reaction,Definition
100.00%,24.39,GLYtm_M,gly_m0 + h_m0 <=> gly_c0 + h_c0
Percent,Flux,Reaction,Definition
42.33%,-10.33,GLYts_M,gly_c0 + h_c0 <=> gly_s0 + h_s0
57.27%,-13.97,GLYtx_M,gly_c0 + h_c0 <=> gly_x0 + h_x0
0.39%,-0.09609,Straw_Biomass_M,0.0321 ala_DASH_L_c0 + 0.0123 arg_DASH_L_c0 + 0.0417 asp_DASH_L_c0 + 33.259 atp_c0 + 2.0789 cellulose_c0 + 0.1721 conoh_c0 + 0.2065 coumoh_c0 + 0.0071 ctp_c0 + 0.0033 cys_DASH_L_c0 + 0.0009 datp_c0 + 0.0007 dctp_c0 + 0.0007 dgtp_c0 + 0.0009 dttp_c0 + 0.00285 fa_os_r0 + 0.0643 fru_DASH_B_c0 + 0.0546 glc_DASH_A_c0 + 0.0385 glu_DASH_L_c0 + 0.0282 gly_c0 + 0.0071 gtp_c0 + 0.057 his_DASH_L_c0 + 0.014 ile_DASH_L_c0 + 0.0234 leu_DASH_L_c0 + 0.0153 lys_DASH_L_c0 + 0.0076 met_DASH_L_c0 + 0.000191 pa_os_r0 + 0.0034 pc_os_r0 + 0.00178 pe_os_r0 + 0.0133 phe_DASH_L_c0 + 0.0225 pro_DASH_L_c0 + 0.00041 ptd1ino_os_r0 + 0.0219 ser_DASH_L_c0 + 0.1475 sinoh_c0 + 0.3249 starch_s0 + 0.1049 sucr_c0 + 0.0192 thr_DASH_L_c0 + 0.0298 triglyc_os_r0 + 0.0016 trp_DASH_L_c0 + 0.0077 tyr_DASH_L_c0 + 0.1256 udparab_c0 + 0.4018 udpg_c0 + 0.0229 udpgal_c0 + 0.5033 udpglcur_c0 + 0.572 udpxyl_c0 + 0.009 utp_c0 + 0.0225 val_DASH_L_c0 --> 33.259 adp_c0 + 1.626 h_c0 + 33.259 pi_c0 + 1.626 udp_c0
